In [ ]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras import utils
import numpy as np
from sklearn.utils import shuffle
from sklearn import preprocessing
import pandas
le = preprocessing.LabelEncoder()
import keras
from keras.callbacks import Callback
import matplotlib.pyplot as plt

import var5


In [ ]:

batch_size = 32
num_epochs = 20
kernel_size = 4
pool_size = 2
conv_depth_1 = 32
conv_depth_2 = 64
drop_prob_1 = 0.25
drop_prob_2 = 0.5
hidden_size = 512

size = 400
img_size = 50


In [ ]:
class CustomCallback(Callback):
    def __init__(self, interval, data, labels):
        super(CustomCallback, self).__init__()
        self.interval = interval
        self.data = data
        self.labels = labels
        self.d = {"epoch": [], "ix": [], "class": [], "acc": [], "err": []}
#         self.acc_fn = accuracy_function
        
    def on_train_end(self, logs=None):
        table = pandas.DataFrame(data=self.d)
        table.to_csv("table.csv")
    
    def on_epoch_end(self, epoch, logs=None):
        n_epochs = self.params["epochs"]
        if epoch % self.interval == 0 or epoch == n_epochs - 1:
            pred = self.model.predict(self.data)
            acc = 1 - abs(self.labels - pred)
            acc= np.mean(acc,axis=1)
            f_loss = keras.losses.get(self.model.loss)
            losses = np.asarray(f_loss(self.labels, pred))
            min_ix = np.argmin(acc)
            min_acc = acc.flatten()[min_ix]
            min_loss = losses[min_ix]
            min_class = self.labels.flatten()[min_ix]
            self.d["epoch"].append(epoch)
            self.d["ix"].append(min_ix)
            self.d["class"].append(int(min_class))
            self.d["acc"].append(min_acc)
            self.d["err"].append(min_loss)
    

In [ ]:
data, labels = var5.gen_data(size,img_size)
# labels.reshape(labels.size)
data, labels = shuffle(data,labels)
num_classes = np.unique(labels).shape[0]
le.fit(labels)
labels = le.transform(labels)
labels = utils.to_categorical(labels, num_classes) # One-hot encode the labels


In [ ]:
inp = Input(shape=(img_size, img_size,1))

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
layer = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
layer = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(layer)
layer = MaxPooling2D(pool_size=(pool_size, pool_size))(layer)
layer = Dropout(drop_prob_1)(layer)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
layer = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
layer = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(layer)
layer = MaxPooling2D(pool_size=(pool_size, pool_size))(layer)
layer = Dropout(drop_prob_1)(layer)


flat = Flatten()(layer)

layer = Dense(hidden_size, activation='relu')(flat)
layer = Dropout(drop_prob_2)(layer)
out = Dense(num_classes, activation='softmax')(layer)


model = Model(inputs=inp, outputs=out)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

H = model.fit(data,labels,
              batch_size=batch_size,
              epochs=num_epochs,
              verbose=1,
              validation_split=.1,
              callbacks = [CustomCallback(num_epochs//5, data, labels)])

model.evaluate(data, labels, verbose=1)